In [3]:
from tensorflow.keras import layers
import tensorflow_addons as tfa
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle as pkl
import keras
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, Model, load_model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input,Bidirectional,Dropout,LayerNormalization,Conv1D, BatchNormalization, \
    multiply, concatenate, Flatten, Activation, dot
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import GaussianMixture
import time
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from keras.layers.core import Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping,ModelCheckpoint
import pydot as pyd
from keras.utils.vis_utils import plot_model, model_to_dot
# keras.utils.vis_utils.pydot = pyd

# Data preprocessing part

In [4]:
def ACE_raw_feature4Delay(df, windowsize=10):
    '''
    df(DataFrame): the path of the processed sensor data
    in our document, there is a csv file which has several traces, so the 'epoch' there means the number of this traces.
    the 'flag' demonstrates if there has attack inside this trace(no matter before, now, or in the future in that trace), and the 'attack_status' demonstrate if the attack already happened before this time slot.
    ACE_1, ACE_2, ACE_3 are the area control errors in different areas.
    'simu_epoch' means the number of this epoch, and 'lastsample_index'is the index of the last sample in that window.
    Return: the original data are cut into windows. And will return the cut windows(DataFrame).
    '''
    Raw_feature2 = []
    simu_epoch = []
    # the attack start index for each epoch, -1 for no attack
    Attack_StartIndex = [] 

    # the index of the last sample for each segment
    LastSample_Index = []
    flag = []
    df_noattack_f = df[df.flag == 0]
    df_attack_f = df[df.flag >0]
    label = []
    ast = []

    # no attack
    for epoch in df_noattack_f.epoch.unique(): 
        df_epoch = df_noattack_f[df_noattack_f.epoch == epoch]
        df_epoch.reset_index(inplace = True)
        StartAttack_index = -1

        # normalized sensors for feature extraction
        df_n20 = np.array(df_epoch[['ACE2_n', 'ACE3_n', 'ACE1_n']])
        for i in range(df_epoch.shape[0] - windowsize): 
            Raw_feature2.append(df_n20[i:i+windowsize].flatten())
            simu_epoch.append(epoch)
            flag.append(0)
            ast.append(0)
            Attack_StartIndex.append(StartAttack_index)
            LastSample_Index.append(i+windowsize-1)

    # has attack
    for epoch in df_attack_f.epoch.unique(): 
        df_epoch = df_attack_f[df_attack_f.epoch == epoch]
        df_epoch.reset_index(inplace = True)
        StartAttack_index = df_epoch[df_epoch.attack_status >0].index[0]  

        # normalized sensors for feature extraction
        df_n20 = np.array(df_epoch[['ACE2_n', 'ACE3_n', 'ACE1_n']])
        df_l21 = np.array(df_epoch[['attack_status']])
        df_m21 = np.array(df_epoch[['flag']])
        for i in range(df_epoch.shape[0] - windowsize):   
            Raw_feature2.append(df_n20[i:i+windowsize].flatten())
            simu_epoch.append(epoch)
            flag.append(df_m21[i + windowsize-1][0])
            ast.append(df_l21[i + windowsize-1][0])
            Attack_StartIndex.append(StartAttack_index)
            LastSample_Index.append(i+windowsize-1)
            
    # save landscape feature 
    df_raw = pd.DataFrame(Raw_feature2)
    df_raw['simu_epoch'] = simu_epoch
    df_raw['LastSample_Index'] = LastSample_Index 
    df_raw['Attack_StartIndex'] = Attack_StartIndex
    df_raw['flag'] = flag
    df_raw['ast'] = ast
    return df_raw

In [6]:
# file for TD attack, csv document
RawFileFold = '/data/delay_fixed.csv'
df_raw = pd.read_csv(RawFileFold)

# keep sensor of ACEs
Scaler = MinMaxScaler()
df_ACE = df_raw.copy()

# normalize sensor data
normal_data = np.array(df_ACE[['Ace_area_2', 'Ace_area_3', 'Ace_area_1', 'attack_status', 'flag']])
df_ACE['ACE2_n'] = normal_data[:, 0]
df_ACE['ACE3_n'] = normal_data[:, 1]
df_ACE['ACE1_n'] = normal_data[:, 2]
df_ACE['attack_status'] = normal_data[:, 3]
df_ACE['flag'] = normal_data[:, 4]
df_ACE['ACE1_n'].shape

(491795,)

In [7]:
# split the whole data into normal epoches and abnormal epoches, perparing for data partation.
df_TDA_noattack = df_ACE[df_ACE.flag == 0]
df_TDA_attack1 = df_ACE[df_ACE.flag== 1]
abnormal_epoch1 = list(df_TDA_attack1.iloc[:, -5].unique())
normal_epoch = list(df_TDA_noattack.iloc[:, -5].unique())

In [8]:
# split the data into train set, test set at 70%, 30%
# 70% abnormal for train
k1 = int(len(abnormal_epoch1)*0.7) 

# 70% normal for train
k0 = int(len(normal_epoch)*0.7) 

# randomly select epoches and append them into the lists
x_train_normal_epochs = []
x_train_normal_epochs.extend(random.sample(normal_epoch, k = k0))
x_train_abnormal_epochs = []
x_train_abnormal_epochs.extend(random.sample(abnormal_epoch1, k = k1))
x_test_normal_epochs = []
for item in df_TDA_noattack.iloc[:, -5].unique():
    if item not in x_train_normal_epochs:
        x_test_normal_epochs.append(item)  
x_test_abnormal_epochs = []
for item in df_TDA_attack1.iloc[:, -5].unique():
    if item not in x_train_abnormal_epochs:
        x_test_abnormal_epochs.append(item)   

# split the TDA dataset to 30% of testset and 70% of train set
df_train_normal = df_TDA_noattack[df_TDA_noattack.iloc[:, -5].isin(x_train_normal_epochs)]
df_test_normal = df_TDA_noattack[df_TDA_noattack.iloc[:, -5].isin(x_test_normal_epochs)]
df_train_abnormal1 = df_TDA_attack1[df_TDA_attack1.iloc[:, -5].isin(x_train_abnormal_epochs)]
df_test_abnormal1 = df_TDA_attack1[df_TDA_attack1.iloc[:, -5].isin(x_test_abnormal_epochs)]
trainable1 = df_train_normal.append(df_train_abnormal1)
testable1 = df_test_normal.append(df_test_abnormal1)

In [9]:
# data perparing
# file for Stealthy FDI attack
RawFileFold = '/data/stealthy_All.csv'
df_raw = pd.read_csv(RawFileFold)

# keep sensor of ACEs
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler()
df_ACE = df_raw.copy()

# normalize sensor data
normal_data = np.array(df_ACE[['Ace_area_2', 'Ace_area_3', 'Ace_area_1', 'attack_status', 'flag']])
df_ACE['ACE2_n'] = normal_data[:, 0]
df_ACE['ACE3_n'] = normal_data[:, 1]
df_ACE['ACE1_n'] = normal_data[:, 2]
df_ACE['attack_status'] = normal_data[:, 3]
df_ACE['flag'] = normal_data[:, 4]
df_ACE['ACE1_n'].shape

(1290000,)

In [10]:
# In our project, the label for TDA is 1 and the label for FDIA is 2, for no attack is 0
df_ACE.epoch += 10000
df_ACE.flag *= 2
b = []
for a in df_ACE.attack_status:
    b.append(2 if a>0 else 0)

# make the FDI attacks have different label
df_ACE.attack_status = pd.DataFrame(b)

In [11]:
# split the whole data into normal epoches and abnormal epoches, perparing for data partation.
df_FDIA_noattack = df_ACE[df_ACE.flag == 0]
df_FDIA_attack1 = df_ACE[df_ACE.flag == 2]
abnormal_epoch1 = list(df_FDIA_attack1.epoch.unique())
normal_epoch = list(df_FDIA_noattack.epoch.unique())

In [12]:
# split the data into train set, test set at 70%, 30%
# 70% abnormal for train
k1 = int(len(abnormal_epoch1)*0.7) 

# 70% normal for train
k0 = int(len(normal_epoch)*0.7) 

# randomly select epoches and append them into the lists
x_train_normal_epochs = []
x_train_normal_epochs.extend(random.sample(normal_epoch, k = k0))
x_train_abnormal_epochs = []
x_train_abnormal_epochs.extend(random.sample(abnormal_epoch1, k = k1))
x_test_normal_epochs = []
for item in df_FDIA_noattack.iloc[:, -5].unique():
    if item not in x_train_normal_epochs:
        x_test_normal_epochs.append(item)  
x_test_abnormal_epochs = []
for item in df_FDIA_attack1.iloc[:, -5].unique():
    if item not in x_train_abnormal_epochs:
        x_test_abnormal_epochs.append(item)   

# split the FDIA dataset to 30% of testset and 70% of train set
df_train_normal = df_FDIA_noattack[df_FDIA_noattack.iloc[:, -5].isin(x_train_normal_epochs)]
df_test_normal = df_FDIA_noattack[df_FDIA_noattack.iloc[:, -5].isin(x_test_normal_epochs)]
df_train_abnormal1 = df_FDIA_attack1[df_FDIA_attack1.iloc[:, -5].isin(x_train_abnormal_epochs)]
df_test_abnormal1 = df_FDIA_attack1[df_FDIA_attack1.iloc[:, -5].isin(x_test_abnormal_epochs)]
trainable2 = df_train_normal.append(df_train_abnormal1)
testable2 = df_test_normal.append(df_test_abnormal1)

In [13]:
# concatenate the data of FDIA and TDA
testable = pd.concat([testable1, testable2])
trainable = pd.concat([trainable1, trainable2])
dftest = ACE_raw_feature4Delay(testable)
dftrain = ACE_raw_feature4Delay(trainable)

In [14]:
# the processing of minmax scaler
Scaler = MinMaxScaler()
a = Scaler.fit(dftrain.iloc[:, :30].values.reshape(dftrain.shape[0]*30, 1))
df1 = a.transform(dftrain.iloc[:, :30].values.reshape(dftrain.shape[0]*30, 1))
dftrain = pd.concat(objs = [pd.DataFrame(df1.reshape(dftrain.shape[0], 30)),dftrain.iloc[:, 30:]],axis = 1,ignore_index = True)
df1 = a.transform(dftest.iloc[:, :30].values.reshape(dftest.shape[0]*30, 1))
dftest = pd.concat(objs = [pd.DataFrame(df1.reshape(dftest.shape[0], 30)),dftest.iloc[:, 30:]],axis = 1,ignore_index = True)

In [15]:
# Only Xtrain will be used for the Autoencoder training
Xtrain = pd.DataFrame(dftrain).iloc[:, :30].values.reshape(dftrain.shape[0], 10, 3)

# AutoEncoder training part

In [16]:
ENC_PROJECTION_DIM = 140
ENC_NUM_HEADS = 3
ENC_LAYERS = 3
NUM_PATCHES = 2
DEC_LAYERS = 2
DEC_NUM_HEADS = 3
LAYER_NORM_EPS = 1e-6
DEC_PROJECTION_DIM = 140
ENC_TRANSFORMER_UNITS = 140
MLPHL = [140]

In [17]:
step = tf.Variable(0, trainable=False)
boundaries = [100000, 150000]
values = [1e-3, 3e-3, 2e-3]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)

In [18]:
def mlp(x, dropout_rate, hidden_units):
    '''
    The MLP layer, included Dense layer and dropout layer
    '''
    for units in hidden_units:
        x = layers.Dense(units, activation = tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [19]:
def create_encoder(num_heads = ENC_NUM_HEADS, num_layers = ENC_LAYERS):
    '''
    The encoder of the autoencoder
    this part will extract the high-dimensional representations out
    '''
    inputs = layers.Input((None, ENC_PROJECTION_DIM))
    x = inputs

    for _ in range(num_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon = LAYER_NORM_EPS)(x)

        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = ENC_PROJECTION_DIM, dropout = 0.1
        )(x1, x1)

        # # Skip connection 1.
        x2 = layers.Add()([attention_output, x])

        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon = LAYER_NORM_EPS)(x2)

        # MLP.
        x3 = mlp(x3, hidden_units = MLPHL, dropout_rate = 0.1)

        # Skip connection 2.
        x = layers.Add()([x3, x2])

    outputs = layers.LayerNormalization(epsilon = LAYER_NORM_EPS)(x)
    return keras.Model(inputs, outputs, name = "ae_encoder")

In [20]:
def create_decoder(
    num_layers = DEC_LAYERS, num_heads = DEC_NUM_HEADS
):
    '''
    The decoder of the autoencoder
    it will reconstruct the original data by using the extracted representations
    '''
    inputs = layers.Input((NUM_PATCHES, ENC_PROJECTION_DIM))
    x = layers.Dense(DEC_PROJECTION_DIM)(inputs)

    for _ in range(num_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=LAYER_NORM_EPS)(x)

        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = DEC_PROJECTION_DIM, dropout=0.1
        )(x1, x1)

        # Skip connection 1.
        x2 = layers.Add()([attention_output, x])

        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon = LAYER_NORM_EPS)(x2)

        # MLP.
        x3 = mlp(x3, hidden_units = MLPHL, dropout_rate = 0.1)

        # Skip connection 2.
        x = layers.Add()([x3, x2])

    x = layers.LayerNormalization(epsilon = 1e-6)(x)
    outputs = layers.Dense(15)(x)

    return keras.Model(inputs, outputs, name = "ae_decoder")

In [21]:
encoder = create_encoder()
decoder = create_decoder()

# the autoencoder
inputs = layers.Input(shape=(10, 3))
# the reshape layer is totally used for the event mechanism, which will get better performance.
x=layers.Reshape((2, 15))(inputs)
embeddings = layers.Dense(140)(x)
encoder_outputs = encoder(embeddings)
decoder_outputs = decoder(encoder_outputs)
decoder_outputs = layers.Reshape((10, 3))(decoder_outputs)
model = Model(inputs = inputs, outputs = decoder_outputs)
opt = tfa.optimizers.AdamW(learning_rate = learning_rate_fn(step), weight_decay = 1e-4)
model.compile(loss = 'mae', optimizer = opt)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 10, 3)]           0         
                                                                 
 reshape (Reshape)           (None, 2, 15)             0         
                                                                 
 dense_7 (Dense)             (None, 2, 140)            2240      
                                                                 
 ae_encoder (Functional)     (None, None, 140)         770980    
                                                                 
 ae_decoder (Functional)     (None, 2, 15)             535935    
                                                                 
 reshape_1 (Reshape)         (None, 10, 3)             0         
                                                                 
Total params: 1,309,155
Trainable params: 1,309,155
Non-train

In [22]:
modelpath = '/model/AE5event'

In [23]:
# for training, each epoch consume around 100s, usually 60 epoch is enough.
epc = 3
es = keras.callbacks.EarlyStopping(
monitor = 'val_loss',
min_delta = 0, 
patience = 35)
history = model.fit(Xtrain,Xtrain, validation_split = 0.15, 
                    epochs = epc, verbose = 1, callbacks = [ModelCheckpoint(filepath = modelpath, monitor = "val_loss", verbose = 1, save_best_only = True), es], 
                    batch_size = 1024,shuffle = True)

Epoch 1/3
936/937 [============================>.] - ETA: 0s - loss: 0.0443
Epoch 1: val_loss improved from inf to 0.02410, saving model to /content/drive/MyDrive/ADSC/MAE/AE5event


937/937 [==============================] - 49s 44ms/step - loss: 0.0442 - val_loss: 0.0241
Epoch 2/3
936/937 [============================>.] - ETA: 0s - loss: 0.0195
Epoch 2: val_loss improved from 0.02410 to 0.02154, saving model to /content/drive/MyDrive/ADSC/MAE/AE5event


937/937 [==============================] - 41s 43ms/step - loss: 0.0195 - val_loss: 0.0215
Epoch 3/3
937/937 [==============================] - ETA: 0s - loss: 0.0145
Epoch 3: val_loss improved from 0.02154 to 0.01861, saving model to /content/drive/MyDrive/ADSC/MAE/AE5event


937/937 [==============================] - 41s 43ms/step - loss: 0.0145 - val_loss: 0.0186


# Evaluation part


In [31]:
def split(normalnum, FDIAnum, TDAnum, dftrain, dftest):
    '''
    extract the dataset for classifier training(Only for classifier)
    normalnum(int): number of normal data for clasifier training
    FDIAnum(int): number of FDIA data for clasifier training
    TDAnum(int): number of TDA data for clasifier training
    dftrain(DataFrame): training dataset
    dftest(DataFrame): testing dataset
    return: the splited data(np.array)
    '''
    dftrain=np.array(dftrain)
    df_TDA_noattack = pd.DataFrame(dftrain[dftrain[:, -1] == 0])
    df_TDA_attack1 = pd.DataFrame(dftrain[dftrain[:, -1] == 1])
    df_TDA_attack2 = pd.DataFrame(dftrain[dftrain[:, -1] == 2])

    #select some labeled data for downstream classifier training
    train_normal = df_TDA_noattack.sample(n = normalnum)
    test_normal = df_TDA_noattack[~df_TDA_noattack.index.isin(train_normal.index)]
    train_abnormal1 = df_TDA_attack1.sample(n = FDIAnum)
    test_abnormal1 = df_TDA_attack1[~df_TDA_attack1.index.isin(train_abnormal1.index)]
    train_abnormal2 = df_TDA_attack2.sample(n = TDAnum)
    test_abnormal2 = df_TDA_attack2[~df_TDA_attack2.index.isin(train_abnormal2.index)]
    train1 = pd.concat([train_normal, train_abnormal1, train_abnormal2])
    test1 = pd.concat([test_normal, test_abnormal1, test_abnormal2])
    dftest = pd.DataFrame(dftest)
    Xtrain = train1.iloc[:, :30].values.reshape(train1.shape[0], 10, 3)
    Ytrain = train1.iloc[:, -1]
    Xtest = dftest.iloc[:, :30].values.reshape(dftest.shape[0], 10, 3)
    Ytest = dftest.iloc[:, -1]
    return Xtrain, Xtest, Ytrain, Ytest

In [34]:
def GMMsplit(GMMnornum, GMMFDIAnum, GMMTDAnum, dftrain):
    '''
    extract the dataset for GMM training(Only for GMM)
    GMMnornum(int): number of normal data for GMM training
    GMMFDIAnum(int): number of FDIA data for GMM training
    GMMTDAnum(int): number of TDA data for GMM training
    dftrain(DataFrame): training dataset
    return: the splited data(np.array)
    '''
    dftrain=np.array(dftrain)
    df_TDA_noattack = pd.DataFrame(dftrain[dftrain[:, -1] == 0])
    df_TDA_attack1 = pd.DataFrame(dftrain[dftrain[:, -1] == 1])
    df_TDA_attack2 = pd.DataFrame(dftrain[dftrain[:, -1] == 2])

    # select some unlabeled data for GMM training, will improve the accuracy by about 1%
    train_normal = df_TDA_noattack.sample(n = GMMnornum)
    test_normal = df_TDA_noattack[~df_TDA_noattack.index.isin(train_normal.index)]
    train_abnormal11 = df_TDA_attack1.sample(n = GMMFDIAnum)
    test_abnormal11 = df_TDA_attack1[~df_TDA_attack1.index.isin(train_abnormal11.index)]
    train_abnormal21 = df_TDA_attack2.sample(n = GMMTDAnum)
    test_abnormal21 = df_TDA_attack2[~df_TDA_attack2.index.isin(train_abnormal21.index)]
    train2 = pd.concat([train_normal, train_abnormal11 ,train_abnormal21])
    XtrainGMM = train2.iloc[:, :30].values.reshape(train2.shape[0], 10, 3)
    return XtrainGMM

In [27]:
normalnum = 200
FDIAnum = 100
TDAnum = 100
GMMnornum = 40000
GMMFDIAnum = 40000
GMMTDAnum = 40000

In [35]:
Xtrain, Xtest, Ytrain, Ytest = split(normalnum, FDIAnum, TDAnum, dftrain, dftest)
XtrainGMM = GMMsplit(GMMnornum, GMMFDIAnum, GMMTDAnum, dftrain)

In [36]:
def acc(Y_test, Ypred):
    '''
    code for Precision and Recall for different attacks' calculation
    Y_test(np_array): The ground true label for the testset.
    Ypred(np_array): The predicted label by the downstream classifier.
    Return: the f1-score for different attacks(float).
    '''
    TN0 = 0
    TP0 = 0
    FP0 = 0
    FN0 = 0
    TN1 = 0
    TP1 = 0
    FP1 = 0
    FN1 = 0
    TN2 = 0
    TP2 = 0
    FP2 = 0
    FN2 = 0
    for i in range(Ypred.shape[0]):
        if Y_test[i] == 1:
            if Ypred[i] == 1:
                TP1 += 1
            else:FN1 += 1
        elif Y_test[i] != 1:
            if Ypred[i] != 1:
                TN1 += 1
            else:FP1 += 1
    for i in range(Ypred.shape[0]):
        if Y_test[i] == 2:
            if Ypred[i] == 2:
                TP2 += 1
            else:FN2 += 1
        elif Y_test[i] != 2:
            if Ypred[i] != 2:
                TN2 += 1
            else:FP2 += 1
    for i in range(Ypred.shape[0]):
        if Y_test[i] == 0:
            if Ypred[i] == 0:
                TP0 += 1
            else:FN0 += 1
        elif Y_test[i] != 0:
            if Ypred[i] != 0:
                TN0 += 1
            else:FP0 += 1
    a = 'for 0 precision/recall/f1:'
    b = 'for Delay attack precision/recall/f1:'
    c = 'for FDI precision/recall/f1:'
    print(a, TP0/(FP0+TP0), TP0/(FN0+TP0), (TP0/(FP0+TP0)+TP0/(FN0+TP0))/2, b, TP1/(FP1+TP1), TP1/(FN1+TP1), (TP1/(FP1+TP1)+TP1/(FN1+TP1))/2, c, TP2/(FP2+TP2), TP2/(FN2+TP2), (TP2/(FP2+TP2)+TP2/(FN2+TP2))/2)

In [37]:
#representation extraction
model = load_model(modelpath)
model.summary()

#We mainly need the first to the forth layers for the representation extractor.
reshape = model.layers[1]
dense = model.layers[2]

# Extract the encoder.
encoder = model.layers[3]
print(encoder)

# Pack as a model.
patches = layers.Input(shape=(10, 3))
x = reshape(patches)
patch_embeddings = dense(x)
encoder_outputs = encoder(patch_embeddings)

# The outputs are the extracted representations 
downstream_model = Model(inputs = patches, outputs = encoder_outputs)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 10, 3)]           0         
                                                                 
 reshape (Reshape)           (None, 2, 15)             0         
                                                                 
 dense_7 (Dense)             (None, 2, 140)            2240      
                                                                 
 ae_encoder (Functional)     (None, None, 140)         770980    
                                                                 
 ae_decoder (Functional)     (None, 2, 15)             535935    
                                                                 
 reshape_1 (Reshape)         (None, 10, 3)             0         
                                                                 
Total params: 1,309,155
Trainable params: 1,309,155
Non-train

In [38]:
# train the GMM and the downstream classifier
pat=layers.Input(shape=(2,140))
m = model.layers[4]
n = model.layers[5]
x1 = m(pat)
x2 = n(x1)
decoder = Model(inputs=pat, outputs=x2)

# and the rest of layers are used for reconstruction error calculation, which is used for the GMM training.
Xpredtra = model.predict(XtrainGMM)

# the reconstruction error
error_train = XtrainGMM-Xpredtra
error_train = error_train.reshape(error_train.shape[0],30)
gm = GaussianMixture(n_components=27, random_state=0).fit(error_train)
Xpredinp = model.predict(Xtrain)
error_inp = Xtrain-Xpredinp
error_inp = error_inp.reshape(error_inp.shape[0],30)

# the error distribution
trainfea = gm.predict(error_inp)
RFinp = downstream_model.predict(Xtrain)
RFinp = RFinp.reshape(RFinp.shape[0], RFinp.shape[1]*RFinp.shape[2])
trainfea = trainfea.reshape(trainfea.shape[0],1)

#New representations.
RFinp1 = np.concatenate([RFinp, trainfea],axis=1)
clf = RandomForestClassifier(n_estimators=1000, random_state=0)
clf.fit(RFinp1, Ytrain)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [39]:
# computation overhead measure and the f1-score calculating
start = time.perf_counter()
RFpred = downstream_model.predict(Xtest)
Xpredtes = decoder.predict(RFpred)
error_test = Xtest-Xpredtes
error_test = error_test.reshape(error_test.shape[0],30)
testfea = gm.predict(error_test)
testfea = testfea.reshape(testfea.shape[0],1)
RFpred = RFpred.reshape(RFpred.shape[0], RFpred.shape[1]*RFpred.shape[2])
RFpred1 = np.concatenate([RFpred, testfea],axis=1)
Ypred = clf.predict(RFpred1)
end = time.perf_counter()
print ((end-start)/Xtest.shape[0])
Y_test = np.array(Ytest)
acc(Y_test, Ypred)

0.0004329291351073239
for 0 precision/recall/f1: 0.9369936679392867 0.9836684568433015 0.9603310623912942 for Delay attack precision/recall/f1: 0.8592912100722779 0.6324649917627677 0.7458781009175228 for FDI precision/recall/f1: 0.9799215750682625 0.9707144455064978 0.9753180102873802
